## Polarization Angle Prediction Pipeline — Development Phases

This pipeline is being built **inside-out**, starting from idealized geometry and working outward toward real sensor data.

### Phase 1 — Geometry & Frame Transformations (current)
**Goal:** Define how a polarization vector is transported through space and turned into a predicted polarization angle.

Assume:
- Known line-of-sight (LOS)
- Known payload attitude
- Known constant vectors (e.g. polarization emission direction)

Focus on:
- Reference frames (payload → world/ENU)
- Vector rotations
- Projection perpendicular to LOS
- Construction of the sky-plane basis
- Extraction of a polarization angle

No real sensor data or hardware noise is involved in this phase.

---

### Phase 2 — Sensor Parsing → Geometry Inputs
**Goal:** Turn raw measurements into the geometric inputs required by Phase 1.

Implement:
- GPS → position → LOS vector (ENU)
- IMU + star camera → payload attitude → rotation matrix or quaternion

The geometry code from Phase 1 is not modified — only the inputs change.

---

### Phase 3 — Lab Measurement of Constants
**Goal:** Measure and fix time-independent quantities.

Includes:
- Polarization emission direction in the payload frame (`p_body`)
- Fixed rotations between payload, IMU, and star camera frames
- Polarimeter reference axis / angle zero-point

These values are inserted into the constants used by the existing pipeline.

---

### Phase 4 — Flight Data & Validation
**Goal:** Apply the full pipeline to real flight data.

Steps:
- Ingest real telemetry
- Compute predicted polarization angles
- Compare to polarimeter measurements
- Analyze residuals, offsets, and systematics

At this stage, discrepancies indicate physical or instrumental effects, not pipeline geometry errors.


# Drone-Based Absolute Polarization Calibration — One-Cell Summary

## Goal
Use a drone-mounted polarized source as an external reference to measure a **global instrumental polarization rotation** of a polarimeter (sky-referenced).

## Lab Calibration (internal parameters only)
In the lab, scan the HWP and fit detector signal vs HWP angle:
d(θ) ≈ A cos(4θ + φ_fit) + C  
For each lab geometry (different drone positions/orientations), extract **one scalar**: the fitted phase φ_fit.

**Known:** drone position & orientation, DRONE→instrument geometry, LOS, HWP angle vs time.  
**Unknown:** p_drone (source polarization direction in the DRONE frame, 2 DOF) and φ_HWP (HWP phase zero, a fixed hardware constant).

Forward model for configuration i:
φ_fit(i) = φ_HWP + 2·ψ_in(i)(p_drone)

Here ψ_in(i) is **not known beforehand**; it is a model prediction computed from a *trial* p_drone and the known geometry for configuration i.  
By fitting φ_fit across multiple non-degenerate geometries, p_drone and φ_HWP are solved **simultaneously**. This step uses no sky/ENU reference and therefore cannot remove any sky-referenced rotation.

## Flight / Absolute Calibration
With p_drone and φ_HWP fixed, fly the drone and compute the predicted sky-plane polarization angle α_pred from geometry. Measure α_meas from the polarimeter. A constant residual
Δα = wrap_180(α_meas − α_pred)
is the **global instrumental polarization rotation**; geometry-dependent residuals indicate more complex systematics.

## Key Conclusions
- HWP start position does **not** create a random offset if HWP phase is fitted.
- φ_HWP is a stable hardware constant (changes only with mechanical changes).
- Fitting p_drone and φ_HWP in the lab does **not** model out global sky-referenced rotation.
- The drone provides a true absolute polarization angle check against the sky.


In [1]:
# we can keep functions in the py file rather than here in the notebook
import function
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
function.imports()

Successfully imported functions.


#### We need to somehow get the emission polarization angle based on the IMU, GPS, and star camera data input streams. 

In [7]:
E = np.array([1.0, 0.0, 0.0])
N = np.array([0.0, 1.0, 0.0])
U = np.array([0.0, 0.0, 1.0])

In [6]:
function.unit

<function function.unit(v: numpy.ndarray) -> numpy.ndarray>

In [8]:
def project_perp(v_world, los_world):
    """Project v_world into the plane perpendicular to los_world."""
    los_hat = function.unit(los_world)
    return v_world - np.dot(v_world, los_hat) * los_hat

## Toy 45-45 Angles by Construction

In [42]:
# line of sight of the drone in ENU, towards the sky
los_ENU = function.unit([0.0, 1.0, 1.0])

# propagation vector from the drone source, in DRONE frame
k_drone = function.unit(np.array([1.0, 0.0, -1.0]))

# polarization axis in the DRONE frame ⟂ k_drone
p_drone = function.unit(np.array([1.0, 0.0, 1.0]))

# assumed rotation matrix that transforms DRONE to ENU (from attitude, no rotation for now)
R_ENU_from_drone = np.eye(3)

# polarization axis in ENU frame
p_ENU = R_ENU_from_drone @ p_drone

# normalized component of polarization oscillation perpandicular to LOS, ENU frame
p_ENU_perpnorm = function.unit(project_perp(p_ENU, los_ENU))

# we define the SKY plane 2D basis, which is what the polarimeter inherently measures in
E_sky_perpnorm = function.unit(project_perp(E, los_ENU))
N_sky_perpnorm = function.unit(np.cross(los_ENU, E_sky_perpnorm))

# find the x and y components of the polarization angle vector in the SKY plane
xcomp = np.dot(p_ENU_perpnorm, E_sky_perpnorm)
ycomp = np.dot(p_ENU_perpnorm, N_sky_perpnorm)

# covert the x and y components to an angle, in SKY plane which matches measurement plane
alpha_pred_rad = np.arctan2(ycomp, xcomp)
alpha_pred_deg = np.degrees(alpha_pred_rad)

# wrap modulo 180 since this is the polarization angle
delta_HWP_deg = 0  # this term must be measured, constant offset caused by zeroed HWP, fit with p_drone. 
apred = (alpha_pred_deg + delta_HWP_deg) % 180.0

## Polarization Angle Projection from Data

In [51]:
# ==============================
# INPUTS THAT MUST COME FROM DATA
# ==============================

# line of sight of the drone in ENU (polarimeter -> drone); you can still use a toy vector here
los_ENU = function.unit([0.0, 1.0, 1.0])  # REQUIRES REAL NUMBERS FROM DRONE FLIGHT DATA

# assumed rotation matrix that transforms DRONE -> ENU (from attitude, no rotation for now)
R_ENU_from_drone = np.eye(3)  # REQUIRES REAL NUMBERS FROM DRONE FLIGHT DATA

# polarization axis in the DRONE frame (emitter axis). It does NOT have to be ⟂ k_drone for all geometries.
p_drone = function.unit(np.array([1.0, 0.0, 1.0]))  # REQUIRES REAL NUMBERS LATER (LAB FIT)

# constant HWP phase offset (system constant). This is the thing you solve for in the LAB FIT.
# IMPORTANT: this is the phase that appears inside cos(4*theta + phi_fit) when you fit the sinusoidal graph.
phi_HWP_deg = 0.0  # REQUIRES REAL NUMBERS LATER (LAB FIT)

# ==============================
# FIXED / DEFINED CONSTANTS
# ==============================

# define the SKY plane 2D basis reference direction in ENU (e.g. East); must be consistent with your convention
E = np.array([1.0, 0.0, 0.0])

# ==============================
# GEOMETRY: CONSISTENT PROPAGATION DIRECTION
# ==============================

# derive the propagation direction consistently from LOS + attitude
# propagation is drone -> polarimeter, so it's opposite of (polarimeter -> drone)
k_ENU = -los_ENU

# rotation ENU -> DRONE is the inverse (transpose) of DRONE -> ENU
R_drone_from_ENU = R_ENU_from_drone.T

# propagation vector of the ray that reaches the polarimeter, expressed in DRONE frame
k_drone = function.unit(R_drone_from_ENU @ k_ENU)

# enforce transversality for the ray that reaches the polarimeter: project p_drone ⟂ k_drone (per geometry)
p_eff_drone = function.unit(project_perp(p_drone, k_drone))

# polarization axis in ENU frame (for this ray)
p_ENU = R_ENU_from_drone @ p_eff_drone

# normalized component of polarization oscillation perpendicular to LOS, ENU frame
p_ENU_perpnorm = function.unit(project_perp(p_ENU, los_ENU))

# ==============================
# SKY-PLANE PROJECTION: PREDICT INCOMING POLARIZATION ANGLE
# ==============================

# we define the SKY plane 2D basis, which is what the polarimeter inherently measures in
E_sky_perpnorm = function.unit(project_perp(E, los_ENU))
N_sky_perpnorm = function.unit(np.cross(los_ENU, E_sky_perpnorm))

# find the x and y components of the polarization vector in the SKY plane
xcomp = np.dot(p_ENU_perpnorm, E_sky_perpnorm)
ycomp = np.dot(p_ENU_perpnorm, N_sky_perpnorm)

# convert the x and y components to an angle, in SKY plane which matches measurement plane
psi_in_rad = np.arctan2(ycomp, xcomp)
psi_in_deg = np.degrees(psi_in_rad)

# wrap modulo 180 since this is a polarization angle (psi == psi + 180)
psi_in_deg = psi_in_deg % 180.0

# ==============================
# PREDICT WHAT YOU READ OFF THE SINUSOIDAL HWP SCAN GRAPH
# ==============================
# If your polarimeter graph is fit as:
#   d(theta) = A * cos(4*theta + phi_fit) + C
# then the fitted phase obeys:
#   phi_fit = phi_HWP + 2*psi_in
#
# NOTE: phi_fit is naturally a phase modulo 360, but many pipelines report it modulo 90 because of 4*theta symmetry.
phi_fit_pred_deg = (phi_HWP_deg + 2.0 * psi_in_deg) % 360.0

# OPTIONAL: if your plotting/fit code reports "psi (mod 90)" (like your screenshot),
# you can convert the phase to an equivalent angle modulo 90.
# (This depends on your plotting convention; keep it only if it matches your fit output.)
psi_graph_pred_deg_mod90 = (phi_fit_pred_deg / 2.0) % 90.0


In [50]:
psi_graph_pred_deg_mod90

54.735610317245346

In [30]:
alpha_pred_deg % 180

144.73561031724535

In [37]:
tol = 1e-10
assert abs(np.linalg.norm(los_ENU) - 1) < tol
assert abs(np.linalg.norm(k_drone) - 1) < tol
assert abs(np.linalg.norm(p_drone) - 1) < tol
assert abs(np.linalg.norm(p_ENU_perpnorm) - 1) < tol
assert abs(np.linalg.norm(E_sky_perpnorm) - 1) < tol
assert abs(np.linalg.norm(N_sky_perpnorm) - 1) < tol
assert abs(np.dot(p_drone, k_drone)) < tol
assert abs(np.dot(p_ENU_perpnorm, los_ENU)) < tol
assert abs(np.dot(E_sky_perpnorm, los_ENU)) < tol
assert abs(np.dot(N_sky_perpnorm, los_ENU)) < tol
assert abs(np.dot(E_sky_perpnorm, N_sky_perpnorm)) < tol
assert abs((xcomp**2 + ycomp**2) - 1.0) < 1e-9
p_test = E_sky_perpnorm
x = np.dot(p_test, E_sky_perpnorm)
y = np.dot(p_test, N_sky_perpnorm)
a = np.degrees(np.arctan2(y, x)) % 180
assert abs(a - 0.0) < 1e-9
p_test = N_sky_perpnorm
x = np.dot(p_test, E_sky_perpnorm)
y = np.dot(p_test, N_sky_perpnorm)
a = np.degrees(np.arctan2(y, x)) % 180
assert abs(a - 90.0) < 1e-9
a1 = (np.degrees(np.arctan2(ycomp, xcomp)) % 180.0)
a2 = (np.degrees(np.arctan2(-ycomp, -xcomp)) % 180.0)
assert abs(a1 - a2) < 1e-9

In [ ]:
assert abs((xcomp**2 + ycomp**2) - 1.0) < 1e-9
